<a href="https://colab.research.google.com/github/lijin-durairaj-code-mode/codes/blob/main/finetuning_optuna_resumeclassification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# !pip install datasets
# !pip install evaluate
# !pip install optuna
# !pip install huggingface_hub
# !pip install optuna==2.10.0

In [ ]:
!pip install optuna-integration[pytorch_lightning]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 811.0/811.0 kB 50.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.9/96.9 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 869.2/869.2 kB 55.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 815.2/815.2 kB 50.1 MB/s eta 0:00:00


In [ ]:
print(optuna.__version__)

2.10.0


In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

from transformers import(
    AutoModel,AutoTokenizer,
    Trainer, TrainingArguments
)
import torch
import torch.nn as nn
from datasets import Dataset
from datasets.dataset_dict import DatasetDict

from huggingface_hub import notebook_login
from tqdm import tqdm
import evaluate
import optuna
from huggingface_hub import HfFolder
import os
from google.colab import userdata
# from optuna.integration import HuggingFacePruningCallback

In [ ]:
print(optuna.__version__)

4.0.0


In [ ]:
!pip install --upgrade optuna pytorch-lightning

In [ ]:
model_name='distilbert-base-uncased'

tokenizer_options={
    'max_length':512,
    'padding':True,
    'truncation':True
}

accuracy_metric = evaluate.load("accuracy")
precision_metric = evaluate.load("precision")
f1score_metric = evaluate.load("f1")

model=AutoModel.from_pretrained(model_name)
tokenizer=AutoTokenizer.from_pretrained(model_name)

num_labels=24

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [ ]:
resume=pd.read_csv('Resume.csv')

unique_labels={d:i for i,d in enumerate(set(resume['Category']))}

train,test=train_test_split(resume,test_size=0.2,random_state=23)

resume_datasetdict=DatasetDict({
    'train':Dataset.from_dict(train),
    'test':Dataset.from_dict(test)
})

In [ ]:
unique_labels

{'CHEF': 0,
 'ENGINEERING': 1,
 'AUTOMOBILE': 2,
 'PUBLIC-RELATIONS': 3,
 'ADVOCATE': 4,
 'FITNESS': 5,
 'FINANCE': 6,
 'HR': 7,
 'AVIATION': 8,
 'CONSULTANT': 9,
 'INFORMATION-TECHNOLOGY': 10,
 'TEACHER': 11,
 'ACCOUNTANT': 12,
 'CONSTRUCTION': 13,
 'HEALTHCARE': 14,
 'BANKING': 15,
 'BUSINESS-DEVELOPMENT': 16,
 'DIGITAL-MEDIA': 17,
 'DESIGNER': 18,
 'AGRICULTURE': 19,
 'APPAREL': 20,
 'SALES': 21,
 'BPO': 22,
 'ARTS': 23}

In [ ]:
def tokenize_data(resume):
  tokenized_data=tokenizer(resume['Resume_str'],**tokenizer_options)
  return tokenized_data

def encode_labels(resume):
  resume['encoded_label']=unique_labels[resume['Category']]
  return resume

def compute_metrics(p):
  pred,labels=p
  preds=np.argmax(pred,axis=1)
  accuracy=accuracy_metric.compute(predictions=preds,references=labels)
  precision_result = precision_metric.compute(predictions=preds, references=labels,average='macro')
  f1_result = f1score_metric.compute(predictions=preds, references=labels,average='macro')
  return{
      'accuracy':accuracy['accuracy'],
      'precision':precision_result['precision'],
      'f1-score':f1_result['f1']
  }


In [ ]:
resume_datasetdict=resume_datasetdict.map(encode_labels)
resume_datasetdict=resume_datasetdict.map(tokenize_data)

resume_datasetdict=resume_datasetdict.rename_column('encoded_label','labels')
resume_datasetdict=resume_datasetdict.select_columns(['input_ids','attention_mask','labels'])

Map:   0%|          | 0/1987 [00:00<?, ? examples/s]

Map:   0%|          | 0/497 [00:00<?, ? examples/s]

Map:   0%|          | 0/1987 [00:00<?, ? examples/s]

Map:   0%|          | 0/497 [00:00<?, ? examples/s]

In [ ]:
class distilbertMultiClass(nn.Module):

  def __init__(self,model,num_labels,dropout_rate):
    super(distilbertMultiClass,self).__init__()
    self.model=model
    self.classifier01=nn.Linear(model.config.hidden_size,model.config.hidden_size)
    self.tanh01=nn.Tanh()
    self.dropout01=nn.Dropout(dropout_rate)

    self.classifier02=nn.Linear(model.config.hidden_size,num_labels)
    self.tanh02=nn.Tanh()
    self.dropout02=nn.Dropout(dropout_rate)

    self.config=model.config

  def forward(self,input_ids,attention_mask,labels=None):
    outputs=self.model(input_ids,attention_mask)
    pooled_output=outputs[0]
    pooled_output=pooled_output[:,0]
    logits=self.classifier01(pooled_output)
    logits=self.tanh01(logits)
    logits=self.dropout01(logits)

    logits=self.classifier02(logits)
    logits=self.tanh02(logits)
    logits=self.dropout02(logits)

    loss=None
    if labels is not None:
      loss_fn=nn.CrossEntropyLoss()
      loss=loss_fn(logits.view(-1,self.classifier02.out_features),labels.view(-1))
    return (loss,logits) if loss is not None else logits

In [ ]:
print('hello world!!!')

hello world!!!


# hyper parameter tuning using optuna

In [ ]:
def objectives(trail):
  try:
    learning_rate=trail.suggest_float('learning_rate',1e-5,2e-5,log=True)
    num_train_epochs=trail.suggest_int('train_epochs',1,5)
    warmup_steps=trail.suggest_int('warmup_step',300,1000)
    weight_decay=trail.suggest_int('weight_decay',0.001,0.05)
    dropout_layer=trail.suggest_int('dropout_layer',0.001,0.5)
    size_per_batch=trail.suggest_int('training_size',1,50)

    #declare model
    distil_model=distilbertMultiClass(model,num_labels,dropout_layer)

    #define gradient descent

    training_args=TrainingArguments(
      learning_rate=learning_rate,
      warmup_steps=warmup_steps,
      weight_decay=weight_decay,
      num_train_epochs=num_train_epochs,
      per_device_train_batch_size=size_per_batch,
      per_device_eval_batch_size=size_per_batch,
      output_dir='classify_resume_model',
      eval_strategy='epoch'
    )

    trainer=Trainer(
      model=distil_model,
      args=training_args,
        train_dataset=resume_datasetdict['train'],
    eval_dataset=resume_datasetdict['test'],
    tokenizer=tokenizer,
      compute_metrics=compute_metrics,
      #  callbacks=[
      #      PyTorchLightningPruningCallback(trail, monitor='eval_loss')
      #      ]
    )

    trainer.train()
    eval_result=trainer.evaluate()
    return eval_result['eval_loss']

  except Exception as ex:
    print('===error===')
    print(ex)
    return None

In [ ]:
study=optuna.create_study(direction='maximize')
study.optimize(objectives,n_trials=50)

[I 2024-10-09 19:20:24,959] A new study created in memory with name: no-name-75ef9dcc-1e51-4c16-8f10-46babc210347
/usr/local/lib/python3.10/dist-packages/optuna/distributions.py:560: UserWarning: The distribution is specified by [0.001, 0.05] and step=1, but the range is not divisible by `step`. It will be replaced by [0.001, 0.001].
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/optuna/distributions.py:560: UserWarning: The distribution is specified by [0.001, 0.5] and step=1, but the range is not divisible by `step`. It will be replaced by [0.001, 0.001].
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy,Precision,F1-score
1,No log,3.165597,0.052314,0.024808,0.010397


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


# applying the hparam values

In [ ]:
distil_model=distilbertMultiClass(model,num_labels,0.3)

training_args=TrainingArguments(
    learning_rate=1.886221353119542e-05,
    warmup_steps=499,
    weight_decay=0,
    num_train_epochs=1,
    output_dir='classify_resume_model',
    eval_strategy='epoch'
)

trainer=Trainer(
    model=distil_model,
    args=training_args,
      train_dataset=resume_datasetdict['train'],
  eval_dataset=resume_datasetdict['test'],
  tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

trainer.train()

AttributeError: 'distilbertMultiClass' object has no attribute 'classifier2'

# testing

In [ ]:
with_testing_data=resume_datasetdict['test'].remove_columns(['labels'])
with_training_data=resume_datasetdict['train'].remove_columns(['labels'])

In [ ]:
import random
indices=random.sample(range(len(with_testing_data)),3)
td=with_testing_data.select(indices)

In [ ]:
td

Dataset({
    features: ['input_ids', 'attention_mask'],
    num_rows: 3
})

In [ ]:
prediction_test=trainer.predict(td)

In [ ]:
predicted_index=np.argmax(prediction_test[0],axis=1)

In [ ]:
predicted_index

array([14,  0,  1])

In [ ]:
[d for d,i in unique_labels.items() if i in predicted_index]

['HEALTHCARE', 'PUBLIC-RELATIONS', 'ENGINEERING']

# testing for huggingface SPACE

In [ ]:
hf_test_resume=pd.read_csv('Resume.csv')

In [ ]:
hf_resume_list=list(hf_test_resume['Resume_str'][0:2])

In [ ]:
hf_resume_list

["         HR ADMINISTRATOR/MARKETING ASSOCIATE\n\nHR ADMINISTRATOR       Summary     Dedicated Customer Service Manager with 15+ years of experience in Hospitality and Customer Service Management.   Respected builder and leader of customer-focused teams; strives to instill a shared, enthusiastic commitment to customer service.         Highlights         Focused on customer satisfaction  Team management  Marketing savvy  Conflict resolution techniques     Training and development  Skilled multi-tasker  Client relations specialist           Accomplishments      Missouri DOT Supervisor Training Certification  Certified by IHG in Customer Loyalty and Marketing by Segment   Hilton Worldwide General Manager Training Certification  Accomplished Trainer for cross server hospitality systems such as    Hilton OnQ  ,   Micros    Opera PMS   , Fidelio    OPERA    Reservation System (ORS) ,   Holidex    Completed courses and seminars in customer service, sales strategies, inventory control, loss p

In [ ]:
hf_tokenized_data_test=tokenizer.batch_encode_plus(
        hf_resume_list,
        max_length=512,
        padding=True,
        truncation=True,
        return_tensors='pt'
        )

In [ ]:
hf_test_data=Dataset.from_dict({
 'input_ids':hf_tokenized_data_test['input_ids'],
 'attention_mask':hf_tokenized_data_test['attention_mask']
})

In [ ]:
_arr=np.array([[-0.0488, -0.1320, -0.0265, -0.3571, -0.7007, -0.2341,  0.0152,  0.2217,
         -0.0415, -0.1640, -0.3344,  0.0125, -0.3945, -0.2280, -0.1009,  0.1063,
          0.1981, -0.0670,  0.2496, -0.1225,  0.1499, -0.0960,  0.1688, -0.0149]])
_arr.shape


(1, 24)

In [ ]:
hf_test_prediction=trainer.predict(hf_test_data)

In [ ]:
hf_test_prediction[0].shape

(2, 24)

In [ ]:
lst=[]
lst.append(
    {
        'name':'a',
        'profile':'abcd'
    }
)
lst.append(
    {
        'name':'b',
        'profile':'abcd'
    }
)

pd.DataFrame(lst)

,name,profile
0,a,abcd
1,b,abcd


In [ ]:
for i, d in enumerate(['a','b']):
  print(i)

0
1
